An oil company is considering purchasing the <span style="color: red"><b>option</b></span> (<span style="color: red"><b>P</b></span>) an oil field. The oil field can have three quality levels (<span style="color: purple"><b>Q</b></span>): high ($q_1$), medium ($q_2$), and low ($q_3$). The company obviously does not know the "real" qaulity of the field beforehand, but it can provide an estimation (i.e., <span style="color: purple"><b>uncertainty</b></span> ) using historical data and imagery. **It is willing to pay a higher price for the field as its quality increases**.

Before making the purchase decision, the company has the <span style="color: red"><b>option</b></span> (<span style="color: red"><b>T</b></span>) to perform a geological test. This test will have a certain cost and its results (<span style="color: purple"><b>R</b></span>) will not be exact predictions about the quality of the field, but will provide a report on the porosity of the reservoir (high porosity generally indicates greater oil potential). The test will not be infallible, and thus contain a certain degree of <span style="color: purple"><b>uncertainty</b></span>. The test can have two possible outcomes:

* **Pass ($r_1$):** The porosity of the reservoir rock is equal to or greater than 15%, indicating significant oil potential.
* **Fail ($r_2$):** The porosity of the reservoir rock is less than 15%, indicating low oil potential.

<table>
<tr>
  <td>
    <img src="./images_1/rock_porosity.jpg" alt="Rock Porosity examples" width="600">
  </td>
</tr>
<tr>
<i><b>Figure 1.</b> Hydrocarbon reservoir quality in terms of permeability and porosity </i>
</tr>
</table>

The chronological sequence of the decision tree is as follows:

1. The company decides whether or not to perform the geological test.
2. If the test is performed, the results are observed.
3. The company decides whether or not to buy the oil field.

There is still residual uncertainty in the problem that affects utility: <span style="color: purple"><b>What is the actual state of the oil field?</b></span>

It is worth noting that any structural asymmetry is explicitly reflected in the decision tree. For example, the test results will only be obtained if the company decides to perform the test.

In this example, it seems logical for the company to buy the oil field after obtaining a "pass" result, but this is not always the case. It will depend on its specific a priori beliefs about the quality of the land (for example, based on its historical data on oil fields with similar characteristics) and on the intrinsic uncertainty of the test (for example, the test may give a positive result but the field is not actually suitable, or vice versa).

**NOTE: PONER AQUI EL DIAGRAM DEL ARBOL DE DECISION Y DEBAJO COMENTAMOS COMO LO VAMOS A RELLENATR DE FORMA CUANQITATIVA. TEMRINANDO AL FINAL CON EL CARBOL DE DECISION CON VALORES CUANTITATIVOS (TRAS COMENTAR QUE HEMOS APLICADO EL TEOREMA DE BAYES)**

In this case, the quantitative information will be specified explicitly to illustrate these ideas. 

#### The Prior Probability Distribution of Oil Field Quality ($Q$)

This table shows the prior probability distribution of the oil field quality, represented by the variable Q. Prior probability refers to the likelihood of encountering each quality level before any observations are made. These probabilities represent the company's belief on the oil field quality. 

<table>
  <tr>
    <th colspan="2"><span style="color: purple">Q</span></th>
  </tr>
  <tr>
    <td>high</td>
    <td>0.3</td>
  </tr>
  <tr>
    <td>medium</td>
    <td>0.4</td>
  </tr>
  <tr>
    <td>low</td>
    <td>0.3</td>
  </tr>
</table>

We can imagine these probabilities were estimated based on historical information from the oil company's past exploration of similar oil fields. For example, the company could have a classification model that predicts the oil field quality by using [satellite imagery and geographical location data](https://www.satimagingcorp.com/applications/energy/exploration/oil-exploration/).

<table>
  <tr>
  <td>
    <img src="./images_1/oil_field_image.jpg" alt="Oil field image" width="400">
  </td>
    <td>
    <img src="./images_1/oil_field_heatmap.jpg" alt="Oil field heatmap" width="400">
  </td>
  </tr>
</table>

#### The conditional probability distribution of the porosity test result (<span style="color: purple"><b>R</b></span>)

The results of the porosity test are directly related to the actual quality of the oil field (<span style="color: purple"><b>Q</b></span>). In a perfect scenario, the test would be highly accurate:

* If the oil field is of **high** quality (<span style="color: purple"><b>Q</b></span> = high), the test result would be "pass" (<span style="color: purple"><b>R</b></span> = pass) with a probability close to 1.
* If the oil field is of **low** quality (<span style="color: purple"><b>Q</b></span> = low), the test result would be "pass" with a probability close to 0.
* If the oil field is of **medium** quality (<span style="color: purple"><b>Q</b></span> = medium), the test result could be "pass" or "fail" with an equal probability (0.5).


However, real-world tests are not perfect. The table below introduces these measurement imperfections by showing the conditional probability of each test result <span style="color: purple"><b>R</b></span> (pass or fail) given the actual quality of the oil field.

<table>
  <tr>
    <th><span style="color: purple">R</span></th>
    <th>high</th>
    <th>medium</th>
    <th>low</th>      
  </tr>
  <tr>
    <td>pass</td>
    <td>0.9</td>
    <td>0.65</td>
    <td>0.15</td>      
  </tr>
  <tr>
    <td>fail</td>
    <td>0.1</td>
    <td>0.35</td>
    <td>0.85</td> 
  </tr>
</table>

#### The utility table ($U$)

<table>
  <tr>
    <th><span style="color: red">T</span></th>
    <th><span style="color: red">P</span></th>
    <th><span style="color: purple">Q</span></th>
    <th><span style="color: blue">U</span></th>
  </tr>
  <tr>
    <td rowspan="6">do</td>
    <td rowspan="3">pass</td>
    <td>high</td>
    <td>0.85</td>
  </tr>
  <tr>
    <td>medium</td>
    <td>0.43</td>
  </tr>
  <tr>
    <td>low</td>
    <td>0</td>
  </tr>
  <tr>
    <td rowspan="3">fail</td>
    <td>high</td>
    <td>0.25</td>
  </tr>
  <tr>
    <td>medium</td>
    <td>0.25</td>
  </tr>
  <tr>
    <td>low</td>
    <td>0.25</td>
  </tr>
  <tr>
    <td rowspan="6">not_do</td>
    <td rowspan="3">pass</td>
    <td>high</td>
    <td>0.86</td>
  </tr>
  <tr>
    <td>medium</td>
    <td>0.44</td>
  </tr>
  <tr>
    <td>low</td>
    <td>0.01</td>
  </tr>
  <tr>
    <td rowspan="3">fail</td>
    <td>high</td>
    <td>0.26</td>
  </tr>
  <tr>
    <td>medium</td>
    <td>0.26</td>
  </tr>
  <tr>
    <td>low</td>
    <td>0.26</td>
  </tr>
</table>

In [1]:
import pyAgrum